# ML2CPP

## Preparing the dataset

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd

def populate_table(tablename, feature_names):
    iris = datasets.load_iris()
    X = iris.data  
    N = X.shape[0]
    y = iris.target.reshape(N,1)
    k = np.arange(N).reshape(N, 1)
    k_X_y = np.concatenate((k, X, y) , axis=1)
    lTable=pd.DataFrame(k_X_y)
    # print(lTable.head())
    lTable.columns = ['idx'] + feature_names + ['TGT'];
    lTable['TGT'] = lTable['TGT'].apply(int)
    lTable['idx'] = lTable['idx'].apply(int)
    lTable.to_csv(tablename , float_format='%.14g')



In [2]:
metadata = {"primary_key" : "KEY",
            "features" : ['sepal_length_cm', 'sepal_width_cm', 'petal_length_cm', 'petal_width_cm'],
            "targets" : ["TGT"],
            "table" : "iris"}

In [3]:
populate_table("/tmp/iris.csv" , metadata["features"])


In [4]:
df = pd.read_csv("/tmp/iris.csv")
df.sample(12, random_state=1960)

,Unnamed: 0,idx,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,TGT
114,114,114,5.8,2.8,5.1,2.4,2
74,74,74,6.4,2.9,4.3,1.3,1
9,9,9,4.9,3.1,1.5,0.1,0
88,88,88,5.6,3.0,4.1,1.3,1
25,25,25,5.0,3.0,1.6,0.2,0
5,5,5,5.4,3.9,1.7,0.4,0
48,48,48,5.3,3.7,1.5,0.2,0
117,117,117,7.7,3.8,6.7,2.2,2
83,83,83,6.0,2.7,5.1,1.6,1
105,105,105,7.6,3.0,6.6,2.1,2


## Training a Model

In [5]:


# train any scikit model on the iris dataset
from sklearn.preprocessing import OrdinalEncoder
clf = OrdinalEncoder()
clf.fit(df[metadata['features']].values, df[metadata['targets']].values)


OrdinalEncoder()

In [6]:
clf.__dict__

{'categories': 'auto',
 'dtype': numpy.float64,
 'categories_': [array([4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2, 5.3, 5.4, 5.5,
         5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8,
         6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.6, 7.7, 7.9]),
  array([2. , 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3,
         3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4. , 4.1, 4.2, 4.4]),
  array([1. , 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.9, 3. , 3.3, 3.5, 3.6,
         3.7, 3.8, 3.9, 4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9,
         5. , 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.3,
         6.4, 6.6, 6.7, 6.9]),
  array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 1. , 1.1, 1.2, 1.3, 1.4, 1.5, 1.6,
         1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5])]}

## Deploying the Model

In [7]:

def generate_cpp_for_model(model):
    import pickle, json, requests, base64
    b64_data = base64.b64encode(pickle.dumps(model)).decode('utf-8')
    # send the model th the web service
    json_data={"Name":"model_cpp_sample", 
               "PickleData":b64_data , 
               "SQLDialect":"CPP",
               "FeatureNames" : metadata['features']}
    r = requests.post("https://sklearn2sql.herokuapp.com/model", json=json_data)
    content = r.json()
    lCPP = content["model"]["SQLGenrationResult"][0]["SQL"]
    # print(lCPP);
    return lCPP


lCPPCode = generate_cpp_for_model(clf);


In [8]:
print(lCPPCode)

namespace  {

	std::vector<std::string> get_input_names(){
		std::vector<std::string> lFeatures = { "Feature_0", "Feature_1", "Feature_2", "Feature_3" };

		return lFeatures;
	}

	std::vector<std::string> get_output_names(){
		std::vector<std::string> lOutputs = { "Feature_0", "Feature_1", "Feature_2", "Feature_3" };

		return lOutputs;
	}

	tTable compute_features(std::any Feature_0, std::any Feature_1, std::any Feature_2, std::any Feature_3) {

		tTable lTable;

		lTable["Feature_0"] = { ( Feature_0 == 4.3 ? 0 : ( Feature_0 == 4.4 ? 1 : ( Feature_0 == 4.5 ? 2 : ( Feature_0 == 4.6 ? 3 : ( Feature_0 == 4.7 ? 4 : ( Feature_0 == 4.8 ? 5 : ( Feature_0 == 4.9 ? 6 : ( Feature_0 == 5.0 ? 7 : ( Feature_0 == 5.1 ? 8 : ( Feature_0 == 5.2 ? 9 : ( Feature_0 == 5.3 ? 10 : ( Feature_0 == 5.4 ? 11 : ( Feature_0 == 5.5 ? 12 : ( Feature_0 == 5.6 ? 13 : ( Feature_0 == 5.7 ? 14 : ( Feature_0 == 5.8 ? 15 : ( Feature_0 == 5.9 ? 16 : ( Feature_0 == 6.0 ? 17 : ( Feature_0 == 6.1 ? 18 : ( Feature_0 == 6.2 ? 

In [9]:
    def write_text_to_file(iCPPCode, oCPPFile):          
        with open(oCPPFile, "w") as text_file:
            text_file.write(iCPPCode)

    def add_cpp_main_function(iCPPCode, iCSVFile):
        lCPPCode = "#include \"Generic.i\"\n\n"
        lCPPCode = lCPPCode + iCPPCode
        lCPPCode = lCPPCode + "\tint main() {\n"
        lCPPCode = lCPPCode + "\t\tscore_csv_file(\"" + iCSVFile +"\");\n"
        lCPPCode = lCPPCode + "\treturn 0;\n}\n"
        return lCPPCode

    def compile_cpp_code_as_executable(iName):
        import subprocess
        lCommand = ["g++", "-Wall", "-Wno-unused-function", "-std=c++17" , "-g" ,  "-o", iName + ".exe",  iName + ".cpp"]
        print("EXECUTING" , "'" + " ".join(lCommand) + "'")
        result = subprocess.check_output(lCommand)
        # print(result)

    def execute_cpp_model(iName, iCSVFile):
        import subprocess
        result2 = subprocess.check_output([iName + ".exe",  iCSVFile])
        result2 = result2.decode()
        print(result2[:1000])
        print(result2[-1000:])
        return result2
        
    def execute_cpp_code(iCPPCode, iCSVFile):
        lName = "/tmp/sklearn2sql_cpp_" + str(id(clf));
        lCPPCode = add_cpp_main_function(iCPPCode, iCSVFile)
        write_text_to_file(lCPPCode, lName + ".cpp")
        compile_cpp_code_as_executable(lName)
        result = execute_cpp_model(lName, iCSVFile)
        write_text_to_file(str(result), lName + ".out")
        return lName + ".out"


In [10]:
populate_table("/tmp/iris2.csv" , ["Feature_0", "Feature_1", "Feature_2", "Feature_3"])
lCPPOutput = execute_cpp_code(lCPPCode , "/tmp/iris2.csv")
cpp_output = pd.read_csv(lCPPOutput)

EXECUTING 'g++ -Wall -Wno-unused-function -std=c++17 -g -o /tmp/sklearn2sql_cpp_140056262940704.exe /tmp/sklearn2sql_cpp_140056262940704.cpp'
idx,Feature_0,Feature_1,Feature_2,Feature_3
0,8,14,4,1
1,6,9,4,1
2,4,11,3,1
3,3,10,5,1
4,7,15,4,1
5,11,18,7,3
6,3,13,4,2
7,7,13,5,1
8,1,8,4,1
9,6,10,5,0
10,11,16,5,1
11,5,13,6,1
12,5,9,4,0
13,0,9,1,0
14,15,19,2,1
15,14,22,5,3
16,11,18,3,3
17,8,14,4,2
18,14,17,7,2
19,8,17,5,2
20,11,13,7,1
21,8,16,5,3
22,3,15,0,1
23,8,12,7,4
24,5,13,8,1
25,7,9,6,1
26,7,13,6,3
27,9,14,5,1
28,9,13,4,1
29,4,11,6,1
30,5,10,6,1
31,11,13,5,3
32,9,20,5,0
33,12,21,4,1
34,6,10,5,1
35,7,11,2,1
36,12,14,3,1
37,6,15,4,0
38,1,9,3,1
39,8,13,5,1
40,7,14,3,2
41,2,2,3,2
42,1,11,3,1
43,7,14,6,5
44,8,17,8,3
45,5,9,4,2
46,8,17,6,1
47,3,11,4,1
48,10,16,5,1
49,7,12,4,1
50,27,11,23,10
51,21,11,21,11
52,26,10,25,11
53,12,2,16,9
54,22,7,22,11
55,14,7,21,9
56,20,12,23,12
57,6,3,10,6
58,23,8,22,9
59,9,6,15,10
60,7,0,11,6
61,16,9,18,11
62,17,1,16,6
63,18,8,23,10
64,13,8,12,9
65,24,10,20,10
66

In [11]:
cpp_output.sample(12, random_state=1960)

,idx,Feature_0,Feature_1,Feature_2,Feature_3
114,114,15,7,27,20
74,74,21,8,19,9
9,9,6,10,5,0
88,88,13,9,17,9
25,25,7,9,6,1
5,5,11,18,7,3
48,48,10,16,5,1
117,117,33,17,41,18
83,83,17,6,27,12
105,105,32,9,40,17


In [12]:
skl_outputs = pd.DataFrame()
X = df[metadata['features']].values
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['idx']);

skl_output_transform = pd.DataFrame(clf.transform(X), columns=cpp_output.columns[1:]);
skl_output = pd.concat([skl_output_key, skl_output_transform] , axis=1)
skl_output.sample(12, random_state=1960)

,idx,Feature_0,Feature_1,Feature_2,Feature_3
114,114,15.0,7.0,27.0,20.0
74,74,21.0,8.0,19.0,9.0
9,9,6.0,10.0,5.0,0.0
88,88,13.0,9.0,17.0,9.0
25,25,7.0,9.0,6.0,1.0
5,5,11.0,18.0,7.0,3.0
48,48,10.0,16.0,5.0,1.0
117,117,33.0,17.0,41.0,18.0
83,83,17.0,6.0,27.0,12.0
105,105,32.0,9.0,40.0,17.0


In [13]:
cpp_skl_join = skl_output.join(cpp_output , how='left', on='idx', lsuffix='_skl', rsuffix='_cpp')

In [14]:
cpp_skl_join.sample(12, random_state=1960)

,idx_skl,Feature_0_skl,Feature_1_skl,Feature_2_skl,Feature_3_skl,idx_cpp,Feature_0_cpp,Feature_1_cpp,Feature_2_cpp,Feature_3_cpp
114,114,15.0,7.0,27.0,20.0,114,15,7,27,20
74,74,21.0,8.0,19.0,9.0,74,21,8,19,9
9,9,6.0,10.0,5.0,0.0,9,6,10,5,0
88,88,13.0,9.0,17.0,9.0,88,13,9,17,9
25,25,7.0,9.0,6.0,1.0,25,7,9,6,1
5,5,11.0,18.0,7.0,3.0,5,11,18,7,3
48,48,10.0,16.0,5.0,1.0,48,10,16,5,1
117,117,33.0,17.0,41.0,18.0,117,33,17,41,18
83,83,17.0,6.0,27.0,12.0,83,17,6,27,12
105,105,32.0,9.0,40.0,17.0,105,32,9,40,17


In [15]:
for col in cpp_output.columns:
    lDiff = cpp_skl_join[col + "_skl"] - cpp_skl_join[col + "_cpp"]
    print(lDiff.describe())

count    150.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
count    150.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
count    150.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
count    150.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
count    150.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
